In [15]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation, SegformerDecodeHead
from PIL import Image
import requests
from torchvision.models.segmentation import fcn_resnet50
from torchvision.models import resnet34

In [2]:
feature_extractor = SegformerFeatureExtractor(size=256).from_pretrained("nvidia/segformer-b2-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b2-finetuned-ade-512-512")


Downloading:   0%|          | 0.00/271 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/105M [00:00<?, ?B/s]

In [18]:
fcn_resnet50()

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [19]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/deeplab1.png", "deeplab1.png")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [21]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'fcn_resnet50', pretrained=True)
model.eval()
input_image = Image.open(filename)
input_image = input_image.convert("RGB")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /Users/barissen/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to /Users/barissen/.cache/torch/hub/checkpoints/fcn_resnet50_coco-1167a1af.pth


  0%|          | 0.00/135M [00:00<?, ?B/s]

In [24]:
features = model.backbone(input_batch)

In [29]:
features['out'].shape

torch.Size([1, 2048, 129, 161])

In [30]:
resnet34_feature_extractor = resnet34()
resnet34_feature_extractor.fc = torch.nn.Sequential()


In [31]:
resnet_features = resnet34_feature_extractor(input_batch)

In [32]:
resnet34_feature_extractor(input_batch).shape


torch.Size([1, 512])

In [36]:
x = resnet34_feature_extractor.conv1(input_batch)
x = resnet34_feature_extractor.bn1(x)
x = resnet34_feature_extractor.relu(x)
x = resnet34_feature_extractor.maxpool(x)
x = resnet34_feature_extractor.layer1(x)
x = resnet34_feature_extractor.layer2(x)
# x = resnet34_feature_extractor.layer3(x)
# x = resnet34_feature_extractor.layer4(x)
x.shape

torch.Size([1, 128, 129, 161])